In [41]:
import os
import json
import numpy as np
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from ipywidgets import interact, fixed, IntSlider

In [2]:
numGames = 40

    rawFolderPath = "D:/EUW/ThreatTestData/Ingame/RawIngameThreatTestData"
    rawFolderPath = "D:/EUW/ThreatTestData/Postgame/RawPostGameThreatTestData"

# Ingame Data: Process and Cleaning

In [6]:
%run fetchGameData.ipynb

def cleanAndSaveIngameFiles(rawFolderPath, cleanedFolderPath, columnsToExclude, displayNoZero, eventsToExclude):
    if not os.path.exists(cleanedFolderPath):
        os.makedirs(cleanedFolderPath)

    files = os.listdir(rawFolderPath)
    totalAmountOfFiles = len(files)  # Define totalAmountOfFiles here
    processedFilesCount = 0

    for filename in files:
        rawFilePath = os.path.join(rawFolderPath, filename)
        cleanedFilePath = os.path.join(cleanedFolderPath, filename)

        with open(rawFilePath, "r", encoding="utf-8") as file:
            data = json.load(file)

        cleanedData = cleanData(data, columnsToExclude, displayNoZero, eventsToExclude)

        with open(cleanedFilePath, "w", encoding="utf-8") as file:
            json.dump(cleanedData, file, indent=4, ensure_ascii=False)
        
        processedFilesCount += 1
        if processedFilesCount % 250 == 0:
            print(f"Processed {processedFilesCount} out of {totalAmountOfFiles} files.")

    if processedFilesCount % 250 != 0 or processedFilesCount == totalAmountOfFiles:
        # Print the final count if it doesn't end on a multiple of 250 or if it's the last file
        print(f"Processed {processedFilesCount} out of {totalAmountOfFiles} files.")



eventsToExclude = ['WARD_PLACED', 'SKILL_LEVEL_UP', 'ITEM_DESTROYED', 'WARD_KILL', 'ITEM_SOLD', 'ITEM_UNDO']
        
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']
displayNoZero = True

Cleaned game data has been saved.


In [7]:
rawFolderPath = "D:/EUW/ThreatTestData/Ingame/RawIngameThreatTestData"
cleanedFolderPath = "D:/EUW/ThreatTestData/Ingame/CleanedIngameThreatTestData"
cleanAndSaveIngameFiles(rawFolderPath, cleanedFolderPath, columnsToExclude, displayNoZero, eventsToExclude)

Processed 40 out of 40 files.


In [8]:
def consolidateAndFilterIngameData(directoryPath, outputFilePath):
    consolidatedData = {}
    gamesProcessed = 0

    for filename in os.listdir(directoryPath):
        if gamesProcessed > numGames:
            break

        filePath = os.path.join(directoryPath, filename)
        
        if os.path.isdir(filePath) or not filePath.endswith('.json'):
            print(f"Skipping: {filePath}")  # Debug print for skipping
            continue

        with open(filePath, 'r', encoding='utf-8') as file:
            ingameData = json.load(file)
        
        relevantData = {
            'frameInterval': ingameData['info'].get('frameInterval', 60000),
            'frames': [],
            'gameEnd': None,
            'allBuildingKills': []  # List to store all BUILDING_KILL events
        }
        
        gameDuration = 0

        for frame in ingameData['info'].get('frames', []):
            newFrame = {'participantFrames': {}, 'events': []}
            
            # Process participant frames for IDs 3 and 8
            for participantId, participantInfo in frame.get('participantFrames', {}).items():
                if int(participantId) in [3, 8]:
                    newFrame['participantFrames'][participantId] = participantInfo

            # Process all BUILDING_KILL events and add them to the allBuildingKills list
            for event in frame.get('events', []):
                if event['type'] == 'BUILDING_KILL':
                    relevantData['allBuildingKills'].append(event)

            # Process CHAMPION_KILL and ELITE_MONSTER_KILL events based on participation by IDs 3 or 8
            for event in frame.get('events', []):
                if event['type'] in ['CHAMPION_KILL', 'ELITE_MONSTER_KILL']:
                    participants = [event.get('killerId')] + event.get('assistingParticipantIds', [])
                    if 'victimId' in event:
                        participants.append(event['victimId'])
                    if any(pid in [3, 8] for pid in participants):
                        newFrame['events'].append(event)

            if newFrame['participantFrames'] or newFrame['events']:
                relevantData['frames'].append(newFrame)
                
            # Look for GAME_END event to capture game duration and winning team
            if 'GAME_END' not in relevantData:  # Ensure we only capture this once per game
                for event in frame.get('events', []):
                    if event['type'] == 'GAME_END':
                        relevantData['gameEnd'] = {
                            'winningTeam': event.get('winningTeam'),
                            'timestamp': event.get('timestamp')
                        }
                        gameDuration = event.get('timestamp')
                        break

        gamesProcessed += 1

        if gameDuration < 300000:  # Skip if game duration is less than 5 minutes
            continue

        matchId = ingameData.get('metadata', {}).get('matchId', f'unknown_{gamesProcessed}')
        consolidatedData[matchId] = relevantData
        
    os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)

    with open(outputFilePath, 'w', encoding='utf-8') as outFile:
        json.dump(consolidatedData, outFile, indent=4)
    
    print(f"Consolidated and filtered data for {gamesProcessed} games has been saved to {outputFilePath}")

In [9]:
directoryPath = "D:/EUW/ThreatTestData/Ingame/CleanedIngameThreatTestData"
outputFilePath = "D:/EUW/ThreatTestData/Ingame/MidIngameThreatTestData.json"
consolidateAndFilterIngameData(directoryPath, outputFilePath)

Consolidated and filtered data for 40 games has been saved to D:/EUW/ThreatTestData/Ingame/MidIngameThreatTestData.json


In [10]:
def saveStructuredAttributesForIngameData(filePath, numGames, outputFilePath):
    try:
        with open(filePath, 'r') as file:
            data = json.load(file)
    except Exception as e:
        print(f"Failed to read or parse JSON from {filePath}: {e}")
        return

    structuredGamesData = {}
    gamesProcessed = 0

    for gameId, gameData in data.items():
        if gamesProcessed >= numGames:
            break

        structuredDataP3, structuredDataP8 = {'frames': []}, {'frames': []}
        gameEndData = gameData.get('gameEnd', None)
        
        allBuildingKills = gameData.get('allBuildingKills', [])
        
        for frame in gameData.get('frames', []):
            frameDataP3 = {'totalGold': None, 'xp': None, 'events': [], 'position': {}, 'damageStats': {}}
            frameDataP8 = {'totalGold': None, 'xp': None, 'events': [], 'position': {}, 'damageStats': {}}

            # Process data for participant 3
            participantData3 = frame.get('participantFrames', {}).get('3')
            if participantData3:
                frameDataP3['totalGold'] = participantData3.get('totalGold', 0)
                frameDataP3['xp'] = participantData3.get('xp', 0)
                frameDataP3['position'] = participantData3.get('position', {})
                frameDataP3['damageStats'] = participantData3.get('damageStats', {})

            # Process data for participant 8
            participantData8 = frame.get('participantFrames', {}).get('8')
            if participantData8:
                frameDataP8['totalGold'] = participantData8.get('totalGold', 0)
                frameDataP8['xp'] = participantData8.get('xp', 0)
                frameDataP8['position'] = participantData8.get('position', {})
                frameDataP8['damageStats'] = participantData8.get('damageStats', {})

            # Process relevant events for this frame
            for event in frame.get('events', []):
                # CHAMPION_KILL events processing
                if event['type'] == 'CHAMPION_KILL' and (event.get('killerId') in [3, 8] or event.get('victimId') in [3, 8] or set(event.get('assistingParticipantIds', [])).intersection({3, 8})):
                    relevant_event = {
                        'killerId': event.get('killerId'),
                        'victimId': event.get('victimId'),
                        'timestamp': event.get('timestamp'),
                        'position': event.get('position'),
                        'assistingParticipants': event.get('assistingParticipantIds', [])
                    }
                    if 3 in [event.get('killerId'), event.get('victimId')] or 3 in event.get('assistingParticipantIds', []):
                        frameDataP3['events'].append(relevant_event)
                    if 8 in [event.get('killerId'), event.get('victimId')] or 8 in event.get('assistingParticipantIds', []):
                        frameDataP8['events'].append(relevant_event)

                # ELITE_MONSTER_KILL events processing
                if event['type'] == 'ELITE_MONSTER_KILL' and (event.get('killerId') in [3, 8] or set(event.get('assistingParticipantIds', [])).intersection({3,8})):
                    elite_event = {
                        'killerId': event.get('killerId'),
                        'killerTeamId': event.get('killerTeamId'),
                        'monsterType': event.get('monsterType'),
                        'position': event.get('position'),
                        'timestamp': event.get('timestamp'),
                        'assistingParticipants': event.get('assistingParticipantIds', [])
                    }
                    if 3 in [event.get('killerId')] or 3 in event.get('assistingParticipantIds', []):
                        frameDataP3['events'].append(elite_event)
                    if 8 in [event.get('killerId')] or 8 in event.get('assistingParticipantIds', []):
                        frameDataP8['events'].append(elite_event)


            structuredDataP3['frames'].append(frameDataP3)
            structuredDataP8['frames'].append(frameDataP8)

        structuredGamesData[gameId] = {
            '3': structuredDataP3,
            '8': structuredDataP8,
            'allBuildingKills': allBuildingKills,  # Include the allBuildingKills list
            'gameEnd': gameEndData
        }
        gamesProcessed += 1

    try:
        with open(outputFilePath, 'w', encoding='utf-8') as outputFile:
            json.dump(structuredGamesData, outputFile, indent=4)
        print(f"Structured data for the first {numGames} games has been saved to {outputFilePath}")
    except Exception as e:
        print(f"Failed to write structured data to {outputFilePath}: {e}")

In [11]:
inputFilePath = "D:/EUW/ThreatTestData/Ingame/MidIngameThreatTestData.json"
outputFilePath = "D:/EUW/ThreatTestData/Ingame/ProcessedMidIngameThreatTestData.json"
saveStructuredAttributesForIngameData(inputFilePath, numGames, outputFilePath)

Structured data for the first 40 games has been saved to D:/EUW/ThreatTestData/Ingame/ProcessedMidIngameThreatTestData.json


--------------------------------------------------------------------------------------------------------------------------

# Postgame Data: Process and Clean

In [12]:
def filterPostData(data):
    # Define the keys to keep in the info section
    infoKeysToKeep = {'endOfGameResult', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId',
                      'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId',
                      'platformId', 'queueId', 'teams', 'tournamentCode'}
    
    # Define participant keys to keep
    participantKeysToKeep = {'championName', 'damageDealtToBuildings', 'damageDealtToObjectives',
                             'damageDealtToTurrets', 'goldEarned', 'goldSpent', 'individualPosition',
                             'inhibitorTakedowns', 'lane', 'participantId', 'perks', 'puuid', 'riotIdGameName',
                             'riotIdTagline', 'role', 'summonerId', 'summonerName', 'teamEarlySurrendered', 'teamId',
                             'teamPosition', 'totalAllyJungleMinionsKilled', 'totalDamageDealt',
                             'totalDamageDealtToChampions', 'totalDamageTaken', 'totalEnemyJungleMinionsKilled',
                             'turretTakedowns', 'visionScore', 'wardsKilled', 'wardsPlaced', 'win', 'challenges'}

    # Define the keys to keep within the challenges dictionary
    challengesKeysToKeep = {'controlWardsPlaced', 'dragonTakedowns', 'goldPerMinute', 'kda',
                            'killParticipation', 'killsNearEnemyTurret', 'killsUnderOwnTurret',
                            'maxCsAdvantageOnLaneOpponent', 'maxLevelLeadLaneOpponent', 'soloKills',
                            'stealthWardsPlaced', 'takedownOnFirstTurret', 'takedowns', 'teamBaronKills',
                            'teamDamagePercentage', 'teamElderDragonKills', 'teamRiftHeraldKills',
                            'visionScorePerMinute', 'wardTakedowns'}
    
    # Keep entire metadata
    filteredData = {'metadata': data['metadata']}
    
    # Filter info section
    filteredInfo = {key: data['info'][key] for key in data['info'] if key in infoKeysToKeep}
    
    # Filter participants
    filteredParticipants = []
    for participant in data['info']['participants']:
        filteredParticipant = {}
        for key in participant:
            if key in participantKeysToKeep:
                if key == 'challenges':
                    # Special handling for the challenges dictionary
                    challenges = participant[key]
                    filteredChallenges = {ckey: challenges[ckey] for ckey in challenges if ckey in challengesKeysToKeep}
                    filteredParticipant[key] = filteredChallenges
                else:
                    filteredParticipant[key] = participant[key]
        filteredParticipants.append(filteredParticipant)
    
    # Add filtered info and participants to the filteredData
    filteredInfo['participants'] = filteredParticipants
    filteredData['info'] = filteredInfo

    return filteredData

def cleanAndSavePostGameData(rawFolderPath, cleanedFolderPath):
    # Ensure the cleaned data folder exists
    if not os.path.exists(cleanedFolderPath):
        os.makedirs(cleanedFolderPath)

    files = os.listdir(rawFolderPath)
    totalAmountOfFiles = len(files)
    processedFilesCount = 0

    # Iterate through each file in the raw data folder
    for filename in files:
        rawFilePath = os.path.join(rawFolderPath, filename)
        cleanedFilePath = os.path.join(cleanedFolderPath, filename)

        # Read the raw game data
        with open(rawFilePath, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Clean the data using filterPostData
        cleanedData = filterPostData(data)

        # Save the cleaned data
        with open(cleanedFilePath, "w", encoding="utf-8") as file:
            json.dump(cleanedData, file, indent=4, ensure_ascii=False)
        
        processedFilesCount += 1
        if processedFilesCount % 250 == 0:
            print(f"Processed {processedFilesCount} out of {totalAmountOfFiles} files.")

    if processedFilesCount % 250 != 0:
        # Print the final count if it doesn't end on a multiple of 250
        print(f"Processed {processedFilesCount} out of {totalAmountOfFiles} files.")

In [13]:
rawFolderPath = "D:/EUW/ThreatTestData/Postgame/RawPostGameThreatTestData"
cleanedFolderPath = "D:/EUW/ThreatTestData/Postgame/CleanedPostGameThreatTestData"
cleanAndSavePostGameData(rawFolderPath, cleanedFolderPath)

Processed 40 out of 40 files.


In [34]:
def extractAndSaveMidPostgameData(directoryPath, outputFilePath):
    lanersData = {}  # Initialize as an empty dictionary
    
    for filename in os.listdir(directoryPath):
        filePath = os.path.join(directoryPath, filename)
        
        with open(filePath, 'r', encoding='utf-8') as file:
            matchData = json.load(file)
            
            if matchData['info']['gameDuration'] < 300:
                continue
            
            matchId = matchData['metadata']['matchId']
            
            for participant in matchData['info']['participants']:
                if participant.get('teamPosition', '').lower() == "middle":
                    lanersData[matchId] = {  # Assign laner data to its matchId
                        'damageDealtToObjectives': participant.get('damageDealtToObjectives', 0),
                        'goldEarned': participant.get('goldEarned', 0),
                        'totalDamageDealt': participant.get('totalDamageDealt', 0),
                        'totalDamageDealtToChampions': participant.get('totalDamageDealtToChampions', 0),
                        'visionScore': participant.get('visionScore', 0),
                        'turretTakedowns': participant.get('turretTakedowns', 0),
                        'wardsPlaced': participant.get('wardsPlaced', 0),
                        'wardsKilled': participant.get('wardsKilled', 0)
                    }
                    break  # Assuming only one participant per teamPosition

    if lanersData:
        os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
        with open(outputFilePath, 'w', encoding='utf-8') as outputFile:
            json.dump(lanersData, outputFile, indent=4)  # Dump the dictionary to the file
        print(f"Mid lane postgame data saved to {outputFilePath}")
    else:
        print("No mid lane postgame data found.")

In [35]:
directoryPath = "D:/EUW/ThreatTestData/Postgame/CleanedPostGameThreatTestData"
outputFilePath = "D:/EUW/ThreatTestData/Postgame/ProcessedMidPostGameThreatTestData.json"
extractAndSaveMidPostgameData(directoryPath, outputFilePath)

Mid lane postgame data saved to D:/EUW/ThreatTestData/Postgame/ProcessedMidPostGameThreatTestData.json


# Compare and Streamline Code

In [36]:
def identifyAndReportUniqueMatchIDs(file1, file2, matchIdKey='matchId'):
    # Load data from the first JSON file
    with open(file1, 'r') as f1:
        data1 = json.load(f1)
    # Extract match IDs from the list
    matchIDsFile1 = set(item[matchIdKey] for item in data1 if matchIdKey in item)
    
    # Load data from the second JSON file
    with open(file2, 'r') as f2:
        data2 = json.load(f2)
    # Extract match IDs from the list
    matchIDsFile2 = set(item[matchIdKey] for item in data2 if matchIdKey in item)
    
    # Find match IDs that are unique to each file
    uniqueToFile1 = matchIDsFile1 - matchIDsFile2
    uniqueToFile2 = matchIDsFile2 - matchIDsFile1
    
    # Calculate the total amount of unique match IDs
    totalUniqueFile1 = len(uniqueToFile1)
    totalUniqueFile2 = len(uniqueToFile2)
    
    # Print out the unique match IDs and their total count
    if uniqueToFile1:
        print(f"Match IDs unique to {file1}:")
        for matchID in uniqueToFile1:
            print(matchID)
        print(f"Total unique match IDs in {file1}: {totalUniqueFile1}")
    else:
        print(f"No match IDs unique to {file1}")
        
    if uniqueToFile2:
        print(f"Match IDs unique to {file2}:")
        for matchID in uniqueToFile2:
            print(matchID)
        print(f"Total unique match IDs in {file2}: {totalUniqueFile2}")
    else:
        print(f"No match IDs unique to {file2}")

In [37]:
file1 = "D:/EUW/ThreatTestData/Postgame/ProcessedMidPostGameThreatTestData.json"
file2 = "D:/EUW/ThreatTestData/Ingame/ProcessedMidIngameThreatTestData.json"
identifyAndReportUniqueMatchIDs(file1, file2)

No match IDs unique to D:/EUW/ThreatTestData/Postgame/ProcessedMidPostGameThreatTestData.json
No match IDs unique to D:/EUW/ThreatTestData/Ingame/ProcessedMidIngameThreatTestData.json


In [38]:
def countGamesInJSON(filePath):
    with open(filePath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Check if the data is a list or a dictionary and count accordingly
    if isinstance(data, list):
        return len(data)  # Each item in the list represents a game
    elif isinstance(data, dict):
        return len(data.keys())  # Each key in the dictionary represents a game
    else:
        return 0  # In case the data is neither a list nor a dictionary

# File paths
file1 = 'D:/EUW/ThreatTestData/Postgame/ProcessedMidPostGameThreatTestData.json'
file2 = 'D:/EUW/ThreatTestData/Ingame/ProcessedMidIngameThreatTestData.json'

# Count the games in each file
gamesInFile1 = countGamesInJSON(file1)
gamesInFile2 = countGamesInJSON(file2)

print(f"Number of games in {file1}: {gamesInFile1}")
print(f"Number of games in {file2}: {gamesInFile2}")


Number of games in D:/EUW/ThreatTestData/Postgame/ProcessedMidPostGameThreatTestData.json: 40
Number of games in D:/EUW/ThreatTestData/Ingame/ProcessedMidIngameThreatTestData.json: 40


# Data Visualization

In [42]:
def plotGameEventsWithTowers(backgroundImagePath, gameDataPath, matchId, targetMinute):
    plt.figure(figsize=(10, 10))
    plt.title(f'Player Positions at Minute {targetMinute} and {targetMinute-1}')
    
    backgroundImage = mpimg.imread(backgroundImagePath)
    plt.xlim(0, 15000)
    plt.ylim(0, 15000)
    plt.imshow(backgroundImage, extent=[0, 15000, 0, 15000], alpha=0.8)

    # Load game data
    with open(gameDataPath, 'r') as file:
        gamesData = json.load(file)
        
    gameData = gamesData.get(matchId)
    if not gameData:
        print(f"No data found for match ID {matchId}")
        return

    def plot_participant_position(frames, participantId, color, targetMinute):
        # Plot participant positions at the target and previous minutes
        for minute, frame in enumerate(frames):
            if minute in [targetMinute, targetMinute - 1] and 'position' in frame:
                alpha = 1 if minute == targetMinute else 0.5
                plt.text(frame['position']['x'], frame['position']['y'], str(participantId),
                         color=color, fontsize=12, ha='center', va='center', alpha=alpha)

    #TODO: scale code to use "3" and "8" as input instead of hardcode
    plot_participant_position(gameData['3']['frames'], 3, '#069aff', targetMinute)     
    plot_participant_position(gameData['8']['frames'], 8, 'red', targetMinute)
    
    # Plot tower destructions with added debugging prints
    print(f"Plotting towers destroyed on or before minute {targetMinute}:")
    def plot_tower_destructions(allBuildingKills, targetMinute):
        # Plot tower destructions up to and including the target minute
        for tower in allBuildingKills:
            destructionMinute = tower['timestamp'] / 60000
            if destructionMinute <= targetMinute:
                # Assuming tower['position'] contains 'x' and 'y' keys
                print(f"Plotting tower: x={tower['position']['x']}, y={tower['position']['y']}, min={destructionMinute}")
                plt.scatter(tower['position']['x'], tower['position']['y'], color='yellow', marker='d', s=100)
                # Optional: Annotate the tower destruction minute or type
                plt.text(tower['position']['x'], tower['position']['y'] - 500, f"Min {destructionMinute:.0f}",
                         color='white', fontsize=9, ha='center')

    # Ensure to call this function where appropriate in your code, 
    # similar to how you call plot_participant_position
    plot_tower_destructions(gameData.get('allBuildingKills', []), targetMinute)


    # Custom legend
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label='Current Frame (Participant 3)', markerfacecolor='#069aff', markersize=15),
        plt.Line2D([0], [0], marker='o', color='w', label='Current Frame (Participant 8)', markerfacecolor='red', markersize=15),
        plt.Line2D([0], [0], marker='o', color='w', label='Previous Frame (less visible)', markerfacecolor='#069aff', markersize=15, alpha=0.5),
        plt.Line2D([0], [0], color='yellow', marker='d', linestyle='None', markersize=10, label='Tower Destroyed')
    ]
    plt.legend(handles=legend_elements, loc='upper right')
    plt.axis('off')
    plt.show()

def getGameDuration(gameDataPath, matchId):
    with open(gameDataPath, 'r') as file:
        gamesData = json.load(file)
    gameData = gamesData.get(matchId)
    if gameData and 'gameEnd' in gameData:
        gameDurationMinutes = gameData['gameEnd']['timestamp'] / 60000
        return gameDurationMinutes
    return None

def interactivePlotFunction(gameDataPath, matchId):
    maxMinutes = getGameDuration(gameDataPath, matchId)
    interact(plotGameEventsWithTowers, 
             backgroundImagePath=fixed('../DataProcessing/TestData/lolMap.PNG'),  # Update this path
             gameDataPath=fixed(gameDataPath),
             matchId=fixed(matchId),
             targetMinute=IntSlider(value=5, min=0, max=int(maxMinutes or 45), step=1, description='Minute:', continuous_update=False))

gameDataPath = 'D:/EUW/ThreatTestData/Ingame/ProcessedMidIngameThreatTestData.json'
matchId = 'EUW1_6844839591'  # Replace with your actual match ID

interactivePlotFunction(gameDataPath, matchId)


interactive(children=(IntSlider(value=5, continuous_update=False, description='Minute:', max=24), Output()), _…